In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import collections
import itertools
import numpy as np

tqdm.pandas()

## Dhaka Tribune

In [2]:
df = pd.read_pickle('Data/Processed Data/Networks/DT-network.pkl')

In [3]:
df['entity0'] = df['entity0'].apply(lambda x: "Sheikh Hasina" if x == "Sheikh Hasina," else x)
df['entity1'] = df['entity1'].apply(lambda x: "Sheikh Hasina" if x == "Sheikh Hasina," else x)

In [4]:
df.head()

,_id,timestamp,entity0,type0,entity1,type1,quote,headline,keywords,original_tags
0,{'$oid': '575f79f55388c1567baebb2a'},{'$date': '2016-06-14T01:28:00.000Z'},Bangladesh,location,Amaq News Agency,organization,"Several hours after the killing, SITE ran a re...",Ashram residents remain in panic,"[bangladesh, ashram, monastery, nityanando, mo...",[crime]
1,{'$oid': '575f79f55388c1567baebb2a'},{'$date': '2016-06-14T01:28:00.000Z'},Noni Gopal,person,Nityanando,person,"Noni Gopal Debnath, another resident of the As...",Ashram residents remain in panic,"[bangladesh, ashram, monastery, nityanando, mo...",[crime]
2,{'$oid': '575f79f55388c1567baebb2a'},{'$date': '2016-06-14T01:28:00.000Z'},Dhaka Tribune,organization,Pabna Alamgir Kabir,person,"Superintendent of Police, Pabna Alamgir Kabir ...",Ashram residents remain in panic,"[bangladesh, ashram, monastery, nityanando, mo...",[crime]
3,{'$oid': '575f79f55388c1567baebb2a'},{'$date': '2016-06-14T01:28:00.000Z'},Islami Chhatra Shibir,organization,Munshi Abu,organization,"Acting on tip-off, a team of police arrested A...",Ashram residents remain in panic,"[bangladesh, ashram, monastery, nityanando, mo...",[crime]
4,{'$oid': '575f79f55388c1567baebb2a'},{'$date': '2016-06-14T01:28:00.000Z'},Islami Chhatra Shibir,organization,Abdus Sattar,person,"Acting on tip-off, a team of police arrested A...",Ashram residents remain in panic,"[bangladesh, ashram, monastery, nityanando, mo...",[crime]


In [5]:
# G = nx.from_pandas_edgelist(df, 'entity0', 'entity1', create_using=nx.Graph())

In [6]:
G = nx.Graph()

In [7]:
network_type = 'person'
count = 0
for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    
    if row['type0'] in network_type and row['type1'] in network_type:
        # print(row['entity0'], row['entity1'])
        count += 1
        G.add_edge(row['entity0'], row['entity1'])
        
print(count)

100%|██████████| 247369/247369 [00:14<00:00, 17391.04it/s]

24925


In [8]:
len(G.nodes)

10132

In [9]:
len(G.edges)

16867

In [10]:
nx.density(G)

0.00032863994112966957

In [11]:
nx.is_connected(G)

False

In [12]:
pr_G = nx.pagerank(G)
sorted(pr_G.items(), key = lambda x:x[1], reverse= True)[:20]

[('Sheikh Hasina', 0.008714946541820293),
 ('Khaleda Zia', 0.006527625334761851),
 ('Ershad', 0.0033629463372757836),
 ('Hossain', 0.0023554989532027747),
 ('Rahman', 0.0022351802397646),
 ('Monirul', 0.0020809714059857303),
 ('Khokon', 0.002002982783159693),
 ('Nur Hossain', 0.001861798382496689),
 ('Babul', 0.0018555368742018638),
 ('Tarique Rahman', 0.001762839395529244),
 ('Ziaur Rahman', 0.00155272881742216),
 ('Kamal', 0.0015279512046905861),
 ('Mizanur Rahman', 0.0014902815077797796),
 ('Hannan', 0.0013829262783952648),
 ('Masud', 0.0013666953228968154),
 ('Shamim', 0.001353066577488366),
 ('Mamun', 0.0013335490807444778),
 ('Latif', 0.0013117671578834467),
 ('Kabir', 0.001274719249852543),
 ('Bangabandhu Sheikh Mujibur Rahman', 0.0012723027199051874)]

In [13]:
len(pr_G.items())

10132

In [14]:
G.remove_edges_from(G.selfloop_edges())
sorted(nx.core_number(G).items(), key= lambda x:x[1], reverse=True)[:20]

[('Hasanul Haque', 9),
 ('Sangeeta Imam', 9),
 ('Mahmudul Haque Munshi Gonojagoron Moncho', 9),
 ('Muhammad Zafar Iqbal', 9),
 ('Sarker', 9),
 ('Arefin Siddique', 9),
 ('Nasiruddin Yusuf', 9),
 ('Shahriar Kabir', 9),
 ('Ramendu Majumdar', 9),
 ('Syed Anwar Hossain', 9),
 ('Mohammad Nasim', 8),
 ('Kamal', 8),
 ('Latif', 8),
 ('Mahfuz Anam', 8),
 ('Ruhul Kabir Rizvi', 8),
 ('Nur', 8),
 ('Ziaur Rahman', 8),
 ('Matia Chowdhury', 8),
 ('Abdul Latif', 8),
 ('Kamal Hossain', 8)]

In [15]:
len(nx.core_number(G))

10132

In [16]:
df = pd.read_pickle('Data/DT/DT.pkl')

In [17]:
df.head()

,_id,is_negative,news_crawled_date,news_headline,news_image_urls,news_keywords,news_location,news_ml_tags,news_naive_tags,news_original_tags,news_publish_date,news_reporters,news_text,news_url,newspaper_name,newspaper_url,locations_unique,organizations_unique,persons_unique
0,{'$oid': '575f79f55388c1567baebb2a'},True,{'$date': '2016-06-13T23:28:50.269Z'},Ashram residents remain in panic,[http://www.dhakatribune.com/sites/default/fil...,"[bangladesh, ashram, monastery, nityanando, mo...",Pabna,NaN,[crime],[crime],{'$date': '2016-06-14T01:28:00.000Z'},[[Md Emroz Khandakar]],People who live in Sri Sri Thakur Anukulchandr...,http://www.dhakatribune.com/crime/2016/jun/14/...,Dhaka Tribune,http://www.dhakatribune.com,[Bangladesh],"[morning. Nityanando, Ishwardi Upazila Health ...","[Abdus Sattar, Noni Gopal, Hemayetpur Dham, Pa..."
1,{'$oid': '575f79f85388c1567baebb2b'},True,{'$date': '2016-06-13T23:28:54.437Z'},Bangladeshi abducted from Bandarban border,[http://www.dhakatribune.com/sites/default/fil...,"[naikkangchhari, area, bandarban, woodcutters,...",Bandarban,NaN,[crime],[bangladesh],{'$date': '2016-06-14T01:27:00.000Z'},[[S Bashu Das]],Separatists in Myanmar abducted a Bangladeshi ...,http://www.dhakatribune.com/bangladesh/2016/ju...,Dhaka Tribune,http://www.dhakatribune.com,"[Bandarban, Dochhari, Myanmar]","[Naikkangchhari, Naikkangchhari BGB]","[Hedman, Abul Khayer, Col Azim, Ula Mang Marma]"
2,{'$oid': '575f79fc5388c1567baebb2c'},True,{'$date': '2016-06-13T23:28:57.408Z'},‘Narayanganj killing was pre-planned’,[http://www.dhakatribune.com/sites/default/fil...,"[murder, rab, picked, preplanned, vehicles, ki...",Narayanganj,NaN,[crime],[crime],{'$date': '2016-06-14T01:25:00.000Z'},[[Tanveer Hossain]],Two personnel of the Rapid Action Battalion ye...,http://www.dhakatribune.com/crime/2016/jun/14/...,Dhaka Tribune,http://www.dhakatribune.com,"[Narayganj, Shitalakya, Narayanganj]","[Rapid Action Battalion, Shitalakhya River., N...","[Syed Enayetur Rahman, Arif Hossain, Nazrul, R..."
3,{'$oid': '575f79fe5388c1567baebb2d'},True,{'$date': '2016-06-13T23:29:00.538Z'},Slum kids see no future without birth registra...,[http://www.dhakatribune.com/sites/default/fil...,"[dhaka, school, kids, pay, slum, shabnam, pall...",National,NaN,[accident],[bangladesh],{'$date': '2016-06-14T01:24:00.000Z'},[Mohammad Jamil Khan],Thirteen-year-old Shabnam had big dreams for h...,http://www.dhakatribune.com/bangladesh/2016/ju...,Dhaka Tribune,http://www.dhakatribune.com,"[Dhaka, Pallabi, Mirpur, New Kurmitola]","[NGO, The Dhaka Tribune, the Urban Primary Hea...",[Shabnam]
4,{'$oid': '575f7a015388c1567baebb2e'},True,{'$date': '2016-06-13T23:29:03.326Z'},Mobile-based agro advisory service makes farme...,[http://www.dhakatribune.com/sites/default/fil...,"[advisory, life, grameenphone, service, farmer...",National,NaN,[accident],[bangladesh],{'$date': '2016-06-14T01:22:00.000Z'},[Abu Bakar Siddique],Amela Begum in Rangpur believes she could have...,http://www.dhakatribune.com/bangladesh/2016/ju...,Dhaka Tribune,http://www.dhakatribune.com,[Rangpur],"[Grameenphone, Oxfam Bangladesh’s ICT, Departm...","[Shefali Rani, Hanif]"


In [18]:
count = collections.Counter([item for sublist in df['persons_unique'] for item in sublist])

In [19]:
count.most_common()[:20]

[('Sheikh Hasina', 4543),
 ('Khaleda Zia', 2616),
 ('Bangabandhu Sheikh Mujibur Rahman', 739),
 ('Abdul Hamid', 653),
 ('Mirza Fakhrul Islam Alamgir', 639),
 ('Mizanur Rahman', 551),
 ('Tarique Rahman', 477),
 ('Obaidul Quader', 433),
 ('Motiur Rahman Nizami', 395),
 ('Abul Kalam', 393),
 ('Ershad', 353),
 ('Abul Kalam Azad', 341),
 ('Ziaur Rahman', 334),
 ('Abul Maal Abdul Muhith', 334),
 ('Anwar Hossain', 304),
 ('Habibur Rahman', 299),
 ('Mahbubey Alam', 299),
 ('Nurul Islam Nahid', 288),
 ('Asaduzzaman Khan Kamal', 276),
 ('Shahidul Haque', 264)]

In [20]:
len(count)

42440

In [21]:
# Find number of times entity appear in headlines

# Here's an idea: Add another column to the DF. Check if Person entity exists in headline or not.
# Make a dictionary as a counter like: {"Person a": 1, "Person b": 0} and put it in the new column

# Also, network probably could've been done like this. In a new column, we could've had list of lists with the combinations.

# Find number of times entity appear in article

In [22]:
# a = {"a": 20, "b":10}
# e = {"a":40, "c":15}

# collections.Counter(a) + collections.Counter(e)

In [23]:
df = pd.read_pickle('Data/DT/DT.pkl')

In [24]:
# df['person_in_headline'] = df[['news_headline', 'persons_unique']].apply()

In [25]:
df.head()

,_id,is_negative,news_crawled_date,news_headline,news_image_urls,news_keywords,news_location,news_ml_tags,news_naive_tags,news_original_tags,news_publish_date,news_reporters,news_text,news_url,newspaper_name,newspaper_url,locations_unique,organizations_unique,persons_unique
0,{'$oid': '575f79f55388c1567baebb2a'},True,{'$date': '2016-06-13T23:28:50.269Z'},Ashram residents remain in panic,[http://www.dhakatribune.com/sites/default/fil...,"[bangladesh, ashram, monastery, nityanando, mo...",Pabna,NaN,[crime],[crime],{'$date': '2016-06-14T01:28:00.000Z'},[[Md Emroz Khandakar]],People who live in Sri Sri Thakur Anukulchandr...,http://www.dhakatribune.com/crime/2016/jun/14/...,Dhaka Tribune,http://www.dhakatribune.com,[Bangladesh],"[morning. Nityanando, Ishwardi Upazila Health ...","[Abdus Sattar, Noni Gopal, Hemayetpur Dham, Pa..."
1,{'$oid': '575f79f85388c1567baebb2b'},True,{'$date': '2016-06-13T23:28:54.437Z'},Bangladeshi abducted from Bandarban border,[http://www.dhakatribune.com/sites/default/fil...,"[naikkangchhari, area, bandarban, woodcutters,...",Bandarban,NaN,[crime],[bangladesh],{'$date': '2016-06-14T01:27:00.000Z'},[[S Bashu Das]],Separatists in Myanmar abducted a Bangladeshi ...,http://www.dhakatribune.com/bangladesh/2016/ju...,Dhaka Tribune,http://www.dhakatribune.com,"[Bandarban, Dochhari, Myanmar]","[Naikkangchhari, Naikkangchhari BGB]","[Hedman, Abul Khayer, Col Azim, Ula Mang Marma]"
2,{'$oid': '575f79fc5388c1567baebb2c'},True,{'$date': '2016-06-13T23:28:57.408Z'},‘Narayanganj killing was pre-planned’,[http://www.dhakatribune.com/sites/default/fil...,"[murder, rab, picked, preplanned, vehicles, ki...",Narayanganj,NaN,[crime],[crime],{'$date': '2016-06-14T01:25:00.000Z'},[[Tanveer Hossain]],Two personnel of the Rapid Action Battalion ye...,http://www.dhakatribune.com/crime/2016/jun/14/...,Dhaka Tribune,http://www.dhakatribune.com,"[Narayganj, Shitalakya, Narayanganj]","[Rapid Action Battalion, Shitalakhya River., N...","[Syed Enayetur Rahman, Arif Hossain, Nazrul, R..."
3,{'$oid': '575f79fe5388c1567baebb2d'},True,{'$date': '2016-06-13T23:29:00.538Z'},Slum kids see no future without birth registra...,[http://www.dhakatribune.com/sites/default/fil...,"[dhaka, school, kids, pay, slum, shabnam, pall...",National,NaN,[accident],[bangladesh],{'$date': '2016-06-14T01:24:00.000Z'},[Mohammad Jamil Khan],Thirteen-year-old Shabnam had big dreams for h...,http://www.dhakatribune.com/bangladesh/2016/ju...,Dhaka Tribune,http://www.dhakatribune.com,"[Dhaka, Pallabi, Mirpur, New Kurmitola]","[NGO, The Dhaka Tribune, the Urban Primary Hea...",[Shabnam]
4,{'$oid': '575f7a015388c1567baebb2e'},True,{'$date': '2016-06-13T23:29:03.326Z'},Mobile-based agro advisory service makes farme...,[http://www.dhakatribune.com/sites/default/fil...,"[advisory, life, grameenphone, service, farmer...",National,NaN,[accident],[bangladesh],{'$date': '2016-06-14T01:22:00.000Z'},[Abu Bakar Siddique],Amela Begum in Rangpur believes she could have...,http://www.dhakatribune.com/bangladesh/2016/ju...,Dhaka Tribune,http://www.dhakatribune.com,[Rangpur],"[Grameenphone, Oxfam Bangladesh’s ICT, Departm...","[Shefali Rani, Hanif]"


In [98]:
def person_in_headline(headline, persons):
    if headline:
        headline_dict = {}
        for i in persons:
            if i in headline:
                if i in headline_dict:
                    headline_dict[i] += 1
                else:
                    headline_dict[i] = 1
        return headline_dict
    else:
        return {}

In [27]:
df['person_in_headline'] = np.vectorize(person_in_headline)(df['news_headline'], df['persons_unique'])

In [92]:
def person_in_text(text, persons):
    text_dict = {}
    for person in persons:
        text_dict[person] = text.count(person)
    return text_dict

In [ ]:
df['person_in_text'] = np.vectorize(person_in_text)(df['news_text'], df['persons_unique'])

In [ ]:
count_person_in_text = sum(map(collections.Counter, df['person_in_text'].values.tolist()), collections.Counter())

In [ ]:
df_text = pd.DataFrame.from_dict(count_person_in_text, orient='index')

In [ ]:
count_person_in_headline = sum(map(collections.Counter, df['person_in_headline'].values.tolist()), collections.Counter())

In [ ]:
df_headline = pd.DataFrame.from_dict(count_person_in_headline, orient='index')

In [ ]:
df_headline.shape

In [ ]:
df_text.shape

In [ ]:
df.shape

In [ ]:
total_persons_unique = set(df['persons_unique'].sum())

In [ ]:
len(total_persons_unique)

In [ ]:
df.shape

In [ ]:
pagerank = pd.DataFrame.from_dict(pr_G, orient='index')
pagerank = pagerank.reset_index()
pagerank = pagerank.sort_values(by='index')
pagerank.columns = ['name', 'pr']

In [ ]:
G.remove_edges_from(G.selfloop_edges())
core = nx.core_number(G)

In [ ]:
core_periphery = pd.DataFrame.from_dict(core, orient='index')
core_periphery = core_periphery.reset_index()
core_periphery = core_periphery.sort_values(by='index')
core_periphery.columns = ['name', 'cp']

In [ ]:
count = dict(count)
person_count = pd.DataFrame.from_dict(count, orient='index')
person_count = person_count.reset_index()
person_count = person_count.sort_values(by='index')
person_count.columns = ['name', 'count']

In [ ]:
df_headline = df_headline.reset_index()
df_headline = df_headline.sort_values(by='index')
df_headline.columns = ['name', 'headline_count']

In [ ]:
df_text = df_text.reset_index()
df_text = df_text.sort_values(by='index')
df_text.columns = ['name', 'text_count']

In [ ]:
ml = pd.DataFrame()

In [ ]:
ml = pd.merge(pagerank, core_periphery, on="name")

In [ ]:
ml = pd.merge(ml, person_count, on="name", how="left")

In [ ]:
ml = pd.merge(ml, df_headline, on="name", how="left")

In [ ]:
ml = pd.merge(ml, df_text, on="name", how="left")

In [ ]:
ml.fillna(0, inplace=True)

In [ ]:
ml.drop(ml.index[0:9], inplace=True)
ml.reset_index(drop=True, inplace=True)

In [ ]:
ml.drop(ml.index[-7:], inplace=True)

## New Age

In [194]:
_df1 = pd.read_pickle('Data/Processed Data/Networks/NewAge-network-1.pkl')
_df2 = pd.read_pickle('Data/Processed Data/Networks/NewAge-network-2.pkl')

In [195]:
df = pd.concat([_df1, _df2], ignore_index=True)

In [196]:
df.head()

,_id,timestamp,entity0,type0,entity1,type1,quote,headline,keywords,category
0,118,2016-10-01,Naogaon,location,Sheikh Hasina,person,"""General elections would be held under prime m...",No national polls before 2019: Nasim,"[General elections, Mohammad Nasim, national p...",Bangladesh
1,118,2016-10-01,Naogaon,location,Noujoan,person,"""General elections would be held under prime m...",No national polls before 2019: Nasim,"[General elections, Mohammad Nasim, national p...",Bangladesh
2,118,2016-10-01,Sheikh Hasina,person,Noujoan,person,"""General elections would be held under prime m...",No national polls before 2019: Nasim,"[General elections, Mohammad Nasim, national p...",Bangladesh
3,118,2016-10-01,Workers Party,organization,Advocate Enamul Haque,organization,Workers Party general secretary Fazle Hossain ...,No national polls before 2019: Nasim,"[General elections, Mohammad Nasim, national p...",Bangladesh
4,118,2016-10-01,Workers Party,organization,NAP,organization,Workers Party general secretary Fazle Hossain ...,No national polls before 2019: Nasim,"[General elections, Mohammad Nasim, national p...",Bangladesh


In [197]:
df.shape

(890480, 10)

In [198]:
df = df.iloc[df.astype(str).drop_duplicates().index]

In [163]:
G = nx.Graph()

In [164]:
network_type = 'person'
exclusions = ['Rakhine', 'Test', 'Dhaka', 'Bangla', 'Bangladesh', 'Narayanganj', 'AL', 'Gulshan', 'Dhanmondi', 'Bogra', 'Rongpur', 'Barisal', 'Rangpur', 'Comilla', 'Chelsea', 'Rajshahi', 'Mymensingh', 'Sadar', 'Sylhet', 'Cox', 'Twitter', 'Barca', 'Kolkata', 'Teknaf', 'Ukhiya', 'Bandarban', 'Tripura', 'Abahani']
count = 0
for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    
    if row['type0'] in network_type and row['type1'] in network_type:
        if row['entity0'] not in exclusions and row['entity1'] not in exclusions:
            # print(row['entity0'], row['entity1'])
            count += 1
            G.add_edge(row['entity0'], row['entity1'])
        
print(count)

100%|██████████| 820506/820506 [00:44<00:00, 18359.40it/s]

79369


In [165]:
print(len(G.nodes))
print(len(G.edges))

23874
64591


In [166]:
nx.density(G)

0.0002266573550150283

In [167]:
pr_G = nx.pagerank(G)
sorted(pr_G.items(), key = lambda x:x[1], reverse= True)[:20]

[('Sheikh Hasina', 0.0037001414913652273),
 ('Donald Trump', 0.0031548373939520808),
 ('Clinton', 0.0013093035154407681),
 ('Putin', 0.0012997899963753145),
 ('Khaleda Zia', 0.001168090017825038),
 ('Rahman', 0.001065581201576243),
 ('Suu Kyi', 0.0010376351665081162),
 ('Khan', 0.0010131492923911808),
 ('Ali', 0.000987753025942132),
 ('Quader', 0.0008565130683169675),
 ('Barack Obama', 0.0008293584893295223),
 ('Aung San Suu Kyi', 0.0008197711494204998),
 ('Tofail Ahmed', 0.0007154948183082828),
 ('Bush', 0.0007082834379158775),
 ('Kim', 0.0007014877986340838),
 ('Abdul Hamid', 0.0006913300533997927),
 ('Jamaat', 0.0006796141348578667),
 ('Md', 0.0006772518321823962),
 ('Vladimir Putin', 0.0006442993711699448),
 ('Oscar', 0.0006373263175412664)]

In [168]:
G.remove_edges_from(G.selfloop_edges())
sorted(nx.core_number(G).items(), key= lambda x:x[1], reverse=True)[:20]

[('Subrata Sengupta', 35),
 ('Jahirul Haque', 35),
 ('Chittaranjan Bhuiyan', 35),
 ('Gitosree Chowdhury', 35),
 ('Mahjabin Begum', 35),
 ('Khandker Raju Ahmed', 35),
 ('Abu Bakar Siddique', 35),
 ('Sajeda Khatun', 35),
 ('Niranjan Adhikari', 35),
 ('Hazrat Ali Bayati', 35),
 ('Uma Khan', 35),
 ('Foyez Ahmad', 35),
 ('Mohiuddin Ahmed', 35),
 ('Dilip Kumar', 35),
 ('Firoj Chowdhury', 35),
 ('Sarwar Jahan', 35),
 ('Abu Nowsher', 35),
 ('Shamser Ali Prodhan', 35),
 ('Sabuj Chakrabarty', 35),
 ('Abul Kalam Azad', 35)]

In [169]:
df = pd.read_pickle('Data/New Age/NewAge_ent.pkl')

In [170]:
df.head()

,category,date_published,image,news_content,news_id,newspaper,reporter,tags,title,url,location_entities,organization_entities,person_entities
0,Bangladesh,2016-10-01,"{'captions': [], 'urls': []}",Coordinator of 14-party alliance and health mi...,118,New Age,Bangladesh Sangbad Sangstha,"[General elections, Mohammad Nasim, national p...",No national polls before 2019: Nasim,http://www.newagebd.net/article/118/no-nationa...,[Naogaon],"[BNP, Workers Party, Advocate Enamul Haque, NA...","[Anwar Hossain, Mohammed Nasim, Sheikh Hasina,..."
1,Entertainment,2016-10-01,"{'captions': ['In this July 4, 2015 file photo...","\n\t\t\t\t\t\t\tIn this July 4, 2015 file phot...",83,New Age,Associated Press,"[David Bowie , Rolling Stones]",Concert promoter in $200 million fraud is bip...,http://www.newagebd.net/article/83/concert-pro...,[Indianapolis],"[the Rolling Stones, the Indianapolis Motor Sp...","[Keith Richards, Michael Hughes, Ronnie Wood, ..."
2,Entertainment,2016-10-01,"{'captions': ['In this March 7, 2000 photo, Ri...","\n\t\t\t\t\t\t\tIn this March 7, 2000 photo, R...",86,New Age,Associated Press,"[Oscar Mayer Wiener , Richard Trentlage]",Man who got generations singing Oscar Mayer W...,http://www.newagebd.net/article/86/man-who-got...,"[Fox River Grove, Chicago, Illinois, Ill., Lib...","[McDonald, Oscar Mayer Wiener, Fox River Grove...","[Linda Bruun, Buckle, Richard Trentlage]"
3,Entertainment,2016-10-01,{'captions': ['A 2015 file photo shows The Gre...,\n\t\t\t\t\t\t\tA 2015 file photo shows The Gr...,89,New Age,Associated Press,"[Henri Matisse, National Gallery of Art , Pabl...",National Gallery of Art's East Building set t...,http://www.newagebd.net/article/89/national-ga...,"[The Greek Slave, Washington, DC, The East Bui...","[the National Gallery of Art, AP]","[Henri Matisse, Georgia O'Keefe, Barbara Kruge..."
4,Entertainment,2016-10-01,{'captions': ['This file photo taken on April ...,\n\t\t\t\t\t\t\tThis file photo taken on April...,90,New Age,Agence France-Presse,"[Bob Dylan, European Commission, Michael Jacks...",Sony closes on Michael Jackson's stake in mus...,http://www.newagebd.net/article/90/sony-closes...,"[United States, Jackson, Kanye West]","[The European Commission, the Democratic Natio...","[Pharrell Williams, Paul McCartney, Michael Ja..."


In [171]:
count = collections.Counter([item for sublist in df['person_entities'] for item in sublist])

In [172]:
count.most_common()[:20]

[('Bangladesh', 3010),
 ('Sheikh Hasina', 2513),
 ('Donald Trump', 1871),
 ('Dhaka', 1516),
 ('Khaleda Zia', 995),
 ('Test', 982),
 ('Barack Obama', 668),
 ('Abul Maal Abdul Muhith', 646),
 ('Obaidul Quader', 622),
 ('Rakhine', 611),
 ('Bangabandhu Sheikh Mujibur Rahman', 570),
 ('Sylhet', 561),
 ('Abdul Hamid', 552),
 ('Mirza Fakhrul Islam Alamgir', 537),
 ('Narayanganj', 509),
 ('Narendra Modi', 495),
 ('AL', 433),
 ('Gulshan', 421),
 ('Sakib al Hasan', 413),
 ('Aung San Suu Kyi', 382)]

In [173]:
df['person_in_headline'] = np.vectorize(person_in_headline)(df['title'], df['person_entities'])

In [174]:
df['person_in_text'] = np.vectorize(person_in_text)(df['news_content'], df['person_entities'])

In [175]:
count_person_in_text = sum(map(collections.Counter, tqdm(df['person_in_text'].values.tolist())), collections.Counter())

100%|██████████| 32893/32893 [13:35<00:00, 40.31it/s]


In [176]:
df_text = pd.DataFrame.from_dict(count_person_in_text, orient='index')

In [177]:
count_person_in_headline = sum(map(collections.Counter, tqdm(df['person_in_headline'].values.tolist())), collections.Counter())

100%|██████████| 32893/32893 [00:27<00:00, 1204.59it/s]


In [178]:
df_headline = pd.DataFrame.from_dict(count_person_in_headline, orient='index')

In [179]:
pagerank = pd.DataFrame.from_dict(pr_G, orient='index')
pagerank = pagerank.reset_index()
pagerank = pagerank.sort_values(by='index')
pagerank.columns = ['name', 'pr']

G.remove_edges_from(G.selfloop_edges())
core = nx.core_number(G)

core_periphery = pd.DataFrame.from_dict(core, orient='index')
core_periphery = core_periphery.reset_index()
core_periphery = core_periphery.sort_values(by='index')
core_periphery.columns = ['name', 'cp']

count = dict(count)
person_count = pd.DataFrame.from_dict(count, orient='index')
person_count = person_count.reset_index()
person_count = person_count.sort_values(by='index')
person_count.columns = ['name', 'count']

df_headline = df_headline.reset_index()
df_headline = df_headline.sort_values(by='index')
df_headline.columns = ['name', 'headline_count']

df_text = df_text.reset_index()
df_text = df_text.sort_values(by='index')
df_text.columns = ['name', 'text_count']

In [180]:
ml = pd.DataFrame()
ml = pd.merge(pagerank, core_periphery, on="name")
ml = pd.merge(ml, person_count, on="name", how="left")
ml = pd.merge(ml, df_headline, on="name", how="left")
ml = pd.merge(ml, df_text, on="name", how="left")
ml.fillna(0, inplace=True)

In [183]:
ml.drop(ml.index[0:59], inplace=True)
ml.reset_index(drop=True, inplace=True)

In [184]:
ml.drop(ml.index[-151:], inplace=True)

In [186]:
ml.to_pickle('Data/Processed Data/ML/NA-ml.pkl')

In [187]:
ml.to_csv('Data/Processed Data/ML/NA-ml.csv')

## Daily Star

In [188]:
df = pd.read_csv('Data/Processed Data/Networks/DS-network.tsv', sep='\t')

In [189]:
df.shape

(936225, 10)

In [190]:
df.head()

,_id,timestamp,entity0,type0,entity1,type1,quote,headline,keywords,section
0,{'$oid': '58afca985388c123849ed1bf'},{'$date': '2008-01-01T00:00:00.000Z'},ZIA,location,Zia International Airport,location,"(L-R) Nasir Uddin, Abbas AliArrested Abbas Ali...",NaN,"['stones', 'rab', 'nasir', 'museum', 'took', '...",Front Page
1,{'$oid': '58afca985388c123849ed1bf'},{'$date': '2008-01-01T00:00:00.000Z'},ZIA,location,Nasir Uddin,person,"(L-R) Nasir Uddin, Abbas AliArrested Abbas Ali...",NaN,"['stones', 'rab', 'nasir', 'museum', 'took', '...",Front Page
2,{'$oid': '58afca985388c123849ed1bf'},{'$date': '2008-01-01T00:00:00.000Z'},ZIA,location,ZIA,location,"(L-R) Nasir Uddin, Abbas AliArrested Abbas Ali...",NaN,"['stones', 'rab', 'nasir', 'museum', 'took', '...",Front Page
3,{'$oid': '58afca985388c123849ed1bf'},{'$date': '2008-01-01T00:00:00.000Z'},ZIA,location,Abbas,person,"(L-R) Nasir Uddin, Abbas AliArrested Abbas Ali...",NaN,"['stones', 'rab', 'nasir', 'museum', 'took', '...",Front Page
4,{'$oid': '58afca985388c123849ed1bf'},{'$date': '2008-01-01T00:00:00.000Z'},ZIA,location,Nasir,person,"(L-R) Nasir Uddin, Abbas AliArrested Abbas Ali...",NaN,"['stones', 'rab', 'nasir', 'museum', 'took', '...",Front Page


In [191]:
df.drop_duplicates(inplace=True)

In [81]:
G = nx.Graph()

In [82]:
network_type = 'person'
count = 0
for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    
    if row['type0'] in network_type and row['type1'] in network_type:
        # print(row['entity0'], row['entity1'])
        count += 1
        G.add_edge(row['entity0'], row['entity1'])
        
print(count)

100%|██████████| 914208/914208 [00:53<00:00, 17003.79it/s]

170661


In [83]:
pr_G = nx.pagerank(G)
sorted(pr_G.items(), key = lambda x:x[1], reverse= True)[:20]

[('Khan', 0.0020221291101969684),
 ('Barack Obama', 0.001860386376414882),
 ('Rahman', 0.0016470698180094299),
 ('Ali', 0.0014392535383854592),
 ('Ahmed', 0.0013892710589818484),
 ('Clinton', 0.0013594442199864772),
 ('Hossain', 0.0011943550496432834),
 ('Rab', 0.0011939810349725446),
 ('Gaddafi', 0.0011799060070162553),
 ('Sheikh Hasina', 0.0011410663011670859),
 ('Nazrul', 0.001096829397606835),
 ('Bush', 0.0010825096553345786),
 ('Singh', 0.0010689226742943228),
 ('Musharraf', 0.0008674203692742137),
 ('Assad', 0.0008148937602715176),
 ('Hillary Clinton', 0.0008097963993054694),
 ('Khaleda Zia', 0.0007314108114951348),
 ('Chowdhury', 0.0007280814787891753),
 ('Abbas', 0.0007247929345799015),
 ('Kim', 0.0006925116346644015)]

In [117]:
G.remove_edges_from(G.selfloop_edges())
sorted(nx.core_number(G).items(), key= lambda x:x[1], reverse=True)[:20]

[('Samar Ranjan Sen', 39),
 ('Kishore Parekh', 39),
 ('Gouri Ghosh', 39),
 ('Waheeda', 39),
 ('Aniruddha Ray', 39),
 ('Maulana Syed Asad Madni', 39),
 ('Pranabranjan Ray', 39),
 ('Kulwant Singh Pannu', 39),
 ('Gautam Chattapadhaya', 39),
 ('Shaheed Subedar Mailkiat Singh', 39),
 ('VK Krisna Meon', 39),
 ('Mrinmoyee Bose', 39),
 ('Somnath Hore', 39),
 ('Ashok Ray', 39),
 ('Asghar Ali', 39),
 ('Shukharanjan Sengupta', 39),
 ('Subhash Mukhopadhaya', 39),
 ('Shakti Chattapadhaya', 39),
 ('Pannalal Dasgupta', 39),
 ('Sukhdev Singh Sanhu', 39)]

In [85]:
df = pd.read_pickle('Data/DS/DS.pkl')

In [93]:
df.head()

,_id,about_reporter,bottom_tagline,breadcrumb,category,content,date_crawled,date_published,generated_keywords,generated_summary,...,ner_unique_person,ner_unique_time,news_url,newspaper,reporter,section,sentiment,shoulder,title,top_tagline
0,{'$oid': '58afca955388c123849ed1be'},NaN,None,[],NaN,People's hopes for the best and confidence in ...,{'$date': '2017-02-24T00:54:28.809Z'},{'$date': '2008-01-01T00:00:00.000Z'},"[parliamentary, fiscal, great, unfulfilled, ca...",This reminded the people of past military regi...,...,"[Muhammad Yunus, Shaikh Abdur Rahman, Khaleda ...",[],http://www.thedailystar.net/news-detail-17307,The Daily Star,None,Front Page,NaN,NaN,Great expectations unfulfilled,None
1,{'$oid': '58afca985388c123849ed1bf'},NaN,None,[],NaN,"(L-R) Nasir Uddin, Abbas AliArrested Abbas Ali...",{'$date': '2017-02-24T00:54:31.997Z'},{'$date': '2008-01-01T00:00:00.000Z'},"[stones, rab, nasir, museum, took, abbas, offi...",Abbas and Nasir said they stole the statuettes...,...,"[Nasir Uddin, SMAK Azad, Atiqur Rahman, ZIA, A...",[],http://www.thedailystar.net/news-detail-17312,The Daily Star,None,Front Page,NaN,NaN,None,None
2,{'$oid': '58afca9a5388c123849ed1c0'},NaN,None,[],NaN,Daniel Vettori and Brendon McCullum put on rec...,{'$date': '2017-02-24T00:54:34.466Z'},{'$date': '2008-01-01T00:00:00.000Z'},"[zealand, seven, balls, bangladesh, odi, vetto...",The match was over before the scheduled first ...,...,"[Jacob Oram, Farhad Reza, Sanath Jayasuriya, B...",[],http://www.thedailystar.net/news-detail-17353,The Daily Star,None,Front Page,NaN,NaN,Kiwis maul Tigers,None
3,{'$oid': '58afca9c5388c123849ed1c1'},NaN,Says BB governor,[],NaN,Bangladesh Bank (BB) Governor Salehuddin Ahmed...,{'$date': '2017-02-24T00:54:36.829Z'},{'$date': '2008-01-01T00:00:00.000Z'},"[wont, outgoing, essentials, soon, governor, i...",Bangladesh Bank in the new year will focus on ...,...,"[Cyclone Sidr, Salehuddin Ahmed]",[],http://www.thedailystar.net/news-detail-17325,The Daily Star,None,Front Page,NaN,NaN,Essentials prices won't go down soon,None
4,{'$oid': '58afca9f5388c123849ed1c2'},NaN,None,[],NaN,Justice KM Sobhan died of cardiac arrest at Bi...,{'$date': '2017-02-24T00:54:39.231Z'},{'$date': '2008-01-01T00:00:00.000Z'},"[passes, sons, sobhan, justice, away, shock, k...","Family sources said Justice Sobhan, also the p...",...,"[Ashraful Islam, Sobhan, Zillur, Zillur Rahman...",[],http://www.thedailystar.net/news-detail-17340,The Daily Star,None,Front Page,NaN,NaN,KM Sobhan passes away,None


In [87]:
df.columns

Index(['_id', 'about_reporter', 'bottom_tagline', 'breadcrumb', 'category',
       'content', 'date_crawled', 'date_published', 'generated_keywords',
       'generated_summary', 'id', 'image_captions', 'images', 'ml_tags',
       'ner_location', 'ner_money', 'ner_organization', 'ner_percent',
       'ner_person', 'ner_time', 'ner_unique_location', 'ner_unique_money',
       'ner_unique_organization', 'ner_unique_percent', 'ner_unique_person',
       'ner_unique_time', 'news_url', 'newspaper', 'reporter', 'section',
       'sentiment', 'shoulder', 'title', 'top_tagline'],
      dtype='object')

In [89]:
count = collections.Counter([item for sublist in df['ner_unique_person'] for item in sublist])

In [90]:
count.most_common()[:20]

[('Barack Obama', 4951),
 ('Sheikh Hasina', 3881),
 ('Rabindranath Tagore', 2764),
 ('Khaleda Zia', 2283),
 ('Rahman', 1949),
 ('Bangabandhu Sheikh Mujibur Rahman', 1933),
 ('Rab', 1456),
 ('Manmohan Singh', 1375),
 ('Hossain', 1316),
 ('Nazrul', 1310),
 ('Hillary Clinton', 1257),
 ('Ahmed', 1179),
 ('Ali', 1110),
 ('Mizanur Rahman', 979),
 ('Bashar al-Assad', 943),
 ('Narendra Modi', 942),
 ('Abul Kalam Azad', 909),
 ('Rafiqul Islam', 899),
 ('Khan', 885),
 ('Hamid Karzai', 858)]

In [104]:
df['person_in_headline'] = np.vectorize(person_in_headline)(df['title'], df['ner_unique_person'])

In [105]:
df['person_in_text'] = np.vectorize(person_in_text)(df['content'], df['ner_unique_person'])

In [108]:
# df_text = pd.DataFrame.from_dict(count_person_in_text, orient='index')

In [111]:
count_person_in_headline = sum(map(collections.Counter, tqdm(df['person_in_headline'].values.tolist())), collections.Counter())

100%|██████████| 165236/165236 [07:59<00:00, 344.56it/s]


In [112]:
df_headline = pd.DataFrame.from_dict(count_person_in_headline, orient='index')

In [113]:
count_person_in_text = sum(map(collections.Counter, tqdm(df['person_in_text'].values.tolist())), collections.Counter())

100%|██████████| 165236/165236 [4:35:14<00:00, 10.01it/s]


In [114]:
df_text = pd.DataFrame.from_dict(count_person_in_text, orient='index')

In [116]:
pagerank = pd.DataFrame.from_dict(pr_G, orient='index')
pagerank = pagerank.reset_index()
pagerank = pagerank.sort_values(by='index')
pagerank.columns = ['name', 'pr']

G.remove_edges_from(G.selfloop_edges())
core = nx.core_number(G)

core_periphery = pd.DataFrame.from_dict(core, orient='index')
core_periphery = core_periphery.reset_index()
core_periphery = core_periphery.sort_values(by='index')
core_periphery.columns = ['name', 'cp']

count = dict(count)
person_count = pd.DataFrame.from_dict(count, orient='index')
person_count = person_count.reset_index()
person_count = person_count.sort_values(by='index')
person_count.columns = ['name', 'count']

df_headline = df_headline.reset_index()
df_headline = df_headline.sort_values(by='index')
df_headline.columns = ['name', 'headline_count']

df_text = df_text.reset_index()
df_text = df_text.sort_values(by='index')
df_text.columns = ['name', 'text_count']

In [127]:
ml = pd.DataFrame()
ml = pd.merge(pagerank, core_periphery, on="name")
ml = pd.merge(ml, person_count, on="name", how="left")
ml = pd.merge(ml, df_headline, on="name", how="left")
ml = pd.merge(ml, df_text, on="name", how="left")
ml.fillna(0, inplace=True)

In [128]:
ml.drop(ml.index[0:86], inplace=True)
ml.reset_index(drop=True, inplace=True)

In [139]:
ml.drop(ml.index[-153:], inplace=True)

In [141]:
# ml.to_pickle('Data/Processed Data/ML/DS-ml.pkl')

In [142]:
# ml.to_csv('Data/Processed Data/ML/DS-ml.csv')